In [4]:
using Pkg

Pkg.activate("../Project.toml")

using Random
import .Iterators: cycle, Cycle, take

using Knet
using Knet: deconv4, conv4, unpool, pool, mat, sigm, KnetArray, nll, zeroone, progress, adam!, sgd!, param, param0, dropout, relu, minibatch, Data
import Knet: train!
using JLD2
using CUDA

 Activating environment at `~/Desktop/MSc/repos/yolo-julia/Project.toml`


In [5]:
function Knet.KnetArray(x::CuArray{T,N}) where {T,N}
    p = Base.bitcast(Knet.Cptr, pointer(x))
    k = Knet.KnetPtr(p, sizeof(x), Int(CUDA.device().handle), x)
    KnetArray{T,N}(k, size(x))
end

In [6]:
include("../coco2014.jl")
x, y = load_data(
    "../dataset/images/overfit",
    "../val_labels.json",
    class_file="../class_mappings.json"
);

dtrn = minibatch(x[:, :, :, 1:6], y[1:6], 2; xsize = (416,416,3,:), xtype=Knet.atype(), shuffle=true);
dtst = minibatch(x[:, :, :, 10:11], y[10:11], 2; xsize = (416,416,3,:), xtype=Knet.atype());

xx, yy = first(dtrn);

In [32]:
include("../models.jl")

Random.seed!(1)
darknet = Darknet("../yolov3.cfg"; verbose=false);


@diff a = darknet(xx, yy)

# a = darknet(xx, yy)

T(571.4975861297888)

In [26]:
ps[6:end, :]

80×9 Array{Float32,2}:
  0.103602    0.0177976    0.035117     …  -0.023552     0.0177457
 -0.393775   -0.203886    -0.171658         0.14615      0.209508
 -0.247163   -0.0791879   -0.213194         0.08069      0.171383
 -0.159323    0.0603991    0.0620692        0.0108701    0.0695402
 -0.185067   -0.0693208   -0.0832997        0.120711     0.119796
  0.0695606   0.0793592    0.0862586    …  -0.291346    -0.251553
  0.109128   -0.0291434   -0.00633341       0.206837     0.127968
  0.298275    0.0952674    0.227818        -0.0278094    0.0382443
 -0.415996   -0.121058    -0.233299        -0.277216    -0.274123
  0.0546542   0.104765     0.185619        -0.0125113   -0.130016
  0.176766    0.0354956    0.120616     …  -0.0353053    0.00487171
 -0.114113   -0.0179149   -0.0651365        0.327846     0.302896
  0.529509    0.132299     0.314457        -0.00105564  -0.0288114
  ⋮                                     ⋱               
  0.0142343   0.0215865    0.000179548     -0.367508    

In [29]:
nll(ps[6:end, :], tcls[1])

4.3883734f0

In [56]:
min.(max.(exp.(ps[3:4, :]), 0), 1000)  # custom clamp

2×11 KnetArray{Float32,2}:
 0.894371  0.894262  0.628493  0.658711  …  1.00179   0.760402  0.781808
 1.01587   1.02907   1.0577    1.04478      0.919599  0.973357  1.02486

3×6 Array{Integer,2}:
 0  0  1  1  1  1
 1  0  1  1  1  1
 1  0  0  0  1  0

In [39]:
an = convert(Array{Bool}, a)

3×6 Array{Bool,2}:
 0  0  1  1  1  1
 1  0  1  1  1  1
 1  0  0  0  1  0

In [19]:
vcat(an'...)

18-element Array{Int64,1}:
 0
 0
 1
 1
 1
 1
 1
 0
 1
 1
 1
 1
 1
 0
 0
 0
 1
 0

In [7]:
function train!(model, train_data::Data, test_data::Data;
                  period::Int=4, iters::Int=100, lr=0.15, optimizer=sgd!)  # or optimizer=adam!
    
    train_loss = []
    test_loss = []
    train_acc = []
    test_acc = []
    
    for i in 0:period:iters
        push!(train_loss, model(train_data))
        push!(test_loss, model(test_data))
        
        optimizer(model, take(cycle(train_data), period); lr=lr)
        
        
        println("Iter: ", i, " ", train_loss[end])
    end
    
    return 0:period:iters, train_loss, train_acc, test_loss, test_acc
end



train! (generic function with 2 methods)

In [ ]:

dtrn = minibatch(x[:, :, :, 1:6], y[1:6], 1; xsize = (416,416,3,:), xtype=Knet.atype(), shuffle=true);
dtst = minibatch(x[:, :, :, 10:11], y[10:11], 1; xsize = (416,416,3,:), xtype=Knet.atype());
include("../models.jl")
Random.seed!(1)

darknet = Darknet("../yolov3.cfg"; verbose=false);

model = darknet

iters, trnloss, trnacc, tstloss, tstacc = train!(
    model, dtrn, dtst; 
    period=1, iters=520, lr=1e-2, optimizer=adam!);

In [22]:
@load "first.jld2" darknet

1-element Array{Symbol,1}:
 :darknet

In [23]:
@diff a = darknet(x[:, :, :, 1:2], y[1:2])

T(5.974961309897038)

In [ ]:
plot(iters, trnloss, label="train", xlabel="Iterations", ylabel="Loss")
display(plot!(iters, tstloss, label="test"))

plot(iters, round.(1 .- trnacc, digits=2), label="train", xlabel="Iterations", ylabel="Misclassification error")
display(plot!(iters, round.(1 .- tstacc, digits=2), label="test"))

In [ ]:

include("../models.jl")
Random.seed!(1)

darknet = Darknet("../yolov3.cfg"; verbose=false);

l = darknet(xx, yy)


In [ ]:
sum(1.0 .- giou)

In [ ]:
inter = clamp.(min.(b1_x2, b2_x2) - max.(b1_x1, b2_x1), 0, 1e10) .* 
clamp.(min.(b1_y2, b2_y2) - max.(b1_y1, b2_y1), 0, 1e10);


In [ ]:
pbox[1, :] .- pbox[3, :] ./ 2

In [ ]:
clamp.(exp.(scores), 0, 1e3)

In [ ]:

b = Integer.(t[:, 1])
c = Integer.(t[:, 2])
gxy = t[:, 3:4]
gwh = t[:, 5:6]

In [ ]:
size(yolo_out[1])

In [ ]:
j

In [ ]:
j'

In [ ]:
tt[jj, :]

In [ ]:
tt = repeat(t, 3)

In [ ]:
tt[j]

In [ ]:
at'[jj]

In [ ]:
targets_reshaped .* gain

In [ ]:
wh1r = reshape(wh1, (:, 1, 2))

In [ ]:
wh12 = reshape(prod(wh1r, dims=3), (:, 1))

In [ ]:
wh2

In [ ]:
wh2r = reshape(wh2, (1, :, 2))

In [ ]:
wh22 = reshape(prod(wh2r, dims=3), (1, :))

In [ ]:
wh12 .+ wh22

In [ ]:
reshape(prod(a, dims=3), (3, 13))

In [ ]:
inter = build_targets(yolo_out, yy, darknet)


In [ ]:
targets = nothing 

for (i, t) in enumerate(yy[2:end])
    println(i)

end
gg = hcat(yy[2]...)
# gg = vcat(ones(1, 2), gg)

In [ ]:
length(yy[2])


In [ ]:
include("utils/utils.jl")

